여러개의 정보가 존재할 때 하나의 값을 추측

In [33]:
import torch

Data

In [34]:
X_train = torch.FloatTensor([[73,80,75],
                             [93,88,93],
                             [89,91,90],
                             [96,98,100],
                             [73,66,70]])
y_train = torch.FloatTensor([[152],[185],[180],[196],[142]])

Hypothesis Function: 인공신경망 구조를 나타냄

H(x) = Wx + b (x:vector, W:matrix)

H(x) = w1x1 + w2x2 + w3x3 + b (입력변수 3개 -> weight 3개)

In [35]:
hypothesis = X1_train * w1 + X2_train + w2 + X3_train * w3 + b

NameError: name 'X1_train' is not defined

matmul(): 한번에 계산 _ x의 길이가 바뀌어도 코드를 바꿀 필요가 없다.

In [ ]:
hypothesis = X_train.matmul(W) + b

Cost function: MSE

In [ ]:
cost = torch.mean((hypothesis - y_train)**2)

Model

In [36]:
from torch import optim

In [37]:
W = torch.zeros((3,1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

optimizer

In [38]:
optimizer = optim.SGD([W,b], lr=1e-5)

계산: Hypothesis, cost, gradient descent

In [39]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
  hypothesis = X_train.matmul(W) + b
  cost = torch.mean((hypothesis - y_train) ** 2)
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  print('Epoch{:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
      epoch, nb_epochs, hypothesis.squeeze().detach(),
      cost.item()
  ))

Epoch   0/20 hypothesis: tensor([0., 0., 0., 0., 0.]) Cost: 29661.800781
Epoch   1/20 hypothesis: tensor([67.2578, 80.8397, 79.6523, 86.7394, 61.6605]) Cost: 9298.520508
Epoch   2/20 hypothesis: tensor([104.9128, 126.0990, 124.2466, 135.3015,  96.1821]) Cost: 2915.712402
Epoch   3/20 hypothesis: tensor([125.9942, 151.4381, 149.2133, 162.4896, 115.5097]) Cost: 915.040527
Epoch   4/20 hypothesis: tensor([137.7967, 165.6247, 163.1911, 177.7112, 126.3307]) Cost: 287.936096
Epoch   5/20 hypothesis: tensor([144.4044, 173.5674, 171.0168, 186.2332, 132.3891]) Cost: 91.371063
Epoch   6/20 hypothesis: tensor([148.1035, 178.0143, 175.3980, 191.0042, 135.7812]) Cost: 29.758249
Epoch   7/20 hypothesis: tensor([150.1744, 180.5042, 177.8509, 193.6753, 137.6805]) Cost: 10.445267
Epoch   8/20 hypothesis: tensor([151.3336, 181.8983, 179.2240, 195.1707, 138.7440]) Cost: 4.391237
Epoch   9/20 hypothesis: tensor([151.9824, 182.6789, 179.9928, 196.0079, 139.3396]) Cost: 2.493121
Epoch  10/20 hypothesis: ten

cost 감소, H(x)는 y에 수렴, lr에 따라 발산할수도 있음

nn.Module: 파이토치에서 제공하는 모듈

In [40]:
import torch.nn as nn

class MultivariateLinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 1)

  def forward(self, x):
    return self.linear(x)

hypothesis = model(X_train)

nn.Module을 상속해서 모델을 생성한다.

nn.Linear(3,1)


*   입력 차원: 3
*   출력 차원: 1

forward(): Hypothesis 계산

PyTorch backward(): Gradient 계산



F.mse_loss: torch.nn.functional에서 제공하는 loss function을 사용하여 cost를 계산한다.

In [41]:
import torch.nn.functional as F
cost = F.mse_loss(prediction, y_train)

<ipython-input-41-81e4d6c260bc>:2: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  cost = F.mse_loss(prediction, y_train)


torch.optim 모델 정의하는 부분이 바뀌게 된다.

데이터

In [42]:
X_train = torch.FloatTensor([[73,80,75],
                             [93,88,93],
                             [89,91,90],
                             [96,98,100],
                             [73,66,70]])
y_train = torch.FloatTensor([[152],[185],[180],[196],[142]])

모델 정의

In [43]:
import torch.nn as nn

class MultivariateLinearRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()
    self.linear = nn.Linear(3, 1)

  def forward(self, x):
    return self.linear(x)

model = MultivariateLinearRegressionModel()

optimizer 설정

In [44]:
optimizer = optim.SGD([W,b],lr=1e-5)

계산: Hypothesis, cost, gradient descent

In [46]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
  Hypothesis = model(X_train)
  cost = F.mse_loss(prediction, y_train)
  optimizer.zero_grad()
  cost.backward()
  optimizer.step()

  print('Epoch{:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
      epoch, nb_epochs, hypothesis.squeeze().detach(),
      cost.item()
  ))

<ipython-input-46-c2f4d3fca44c>:4: UserWarning: Using a target size (torch.Size([5, 1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  cost = F.mse_loss(prediction, y_train)


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

Real World의 Dataset은 적어도 수십만개이다.

많은 데이터를 다 학습할 수 없어서 일부분의 데이터로만 학습한다.

전체 데이터를 minibatch로 나누어서 학습한다.

PyTorch Dataset 만들기

torch.utils.data.Dataset 상속

In [47]:
from torch.utils.data import Dataset

__len__(): dataset의 총 data수

__getitem__(): index를 받았을 때, 반환하는 입출력 데이터

In [48]:
class CustomDataset(Dataset):
  def __init__(self):
    self.X_data = ([[73,80,75],
                    [93,88,93],
                    [89,91,90],
                    [96,98,100],
                    [73,66,70]])
    self.y_data = ([[152],[185],[180],[196],[142]])

  def __len__(self):
    return len(self.X_data)

  def __getitem__(self, idx):
    X = torch.FloatTensor(self.X_data[idx])
    y = torch.FloatTensor(self.y_data[idx])

    return X, y

dataset = CustomDataset()

PyTorch DataLoader

In [49]:
from torch.utils.data import DataLoader
dataloader = DataLoader(
    dataset,
    batch_size=2,
    shuffle=True,
)

batch_size = 2
- 각 미니배치의 크기이다. 통상적으로 2의 제곱수로 설정

shuffle=True
- Epoch마다 데이터셋을 섞는다. 데이터가 학습되는 순서를 바꾼다.



In [50]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
  for batch_idx, samples in enumerate(dataloader):
    X_train, y_train = samples

    prediction = model(X_train)

    cost = F.mse_loss(prediction, y_train)

    optimizer.zero_grad()
    cost.backward()
    optimizer.step()


    print('Epoch{:4d}/{} Batch {} Cost: {:.6f}'.format(
      epoch, nb_epochs, batch_idx+1, len(dataloader),
      cost.item()
    ))

Epoch   0/20 Batch 1 Cost: 3.000000
Epoch   0/20 Batch 2 Cost: 3.000000
Epoch   0/20 Batch 3 Cost: 3.000000
Epoch   1/20 Batch 1 Cost: 3.000000
Epoch   1/20 Batch 2 Cost: 3.000000
Epoch   1/20 Batch 3 Cost: 3.000000
Epoch   2/20 Batch 1 Cost: 3.000000
Epoch   2/20 Batch 2 Cost: 3.000000
Epoch   2/20 Batch 3 Cost: 3.000000
Epoch   3/20 Batch 1 Cost: 3.000000
Epoch   3/20 Batch 2 Cost: 3.000000
Epoch   3/20 Batch 3 Cost: 3.000000
Epoch   4/20 Batch 1 Cost: 3.000000
Epoch   4/20 Batch 2 Cost: 3.000000
Epoch   4/20 Batch 3 Cost: 3.000000
Epoch   5/20 Batch 1 Cost: 3.000000
Epoch   5/20 Batch 2 Cost: 3.000000
Epoch   5/20 Batch 3 Cost: 3.000000
Epoch   6/20 Batch 1 Cost: 3.000000
Epoch   6/20 Batch 2 Cost: 3.000000
Epoch   6/20 Batch 3 Cost: 3.000000
Epoch   7/20 Batch 1 Cost: 3.000000
Epoch   7/20 Batch 2 Cost: 3.000000
Epoch   7/20 Batch 3 Cost: 3.000000
Epoch   8/20 Batch 1 Cost: 3.000000
Epoch   8/20 Batch 2 Cost: 3.000000
Epoch   8/20 Batch 3 Cost: 3.000000
Epoch   9/20 Batch 1 Cost: 3